<a href="https://colab.research.google.com/github/demyurg11/demyurg/blob/master/Torrent_To_Google_Drive_Downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Torrent To Google Drive Downloader

**Important Note:** To get more disk space:
> Go to Runtime -> Change Runtime and give GPU as the Hardware Accelerator.  You will get around 384GB to download any torrent you want.

### Install libtorrent and Initialize Session

In [2]:
!apt install python3-libtorrent

import libtorrent as lt

ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3-libtorrent is already the newest version (2.0.5-5).
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.


<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for std::pair<int, int> already registered; second conversion method ignored.
<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for std::pair<libtorrent::aux::strong_typedef<int, libtorrent::aux::piece_index_tag, void>, libtorrent::aux::strong_typedef<unsigned char, libtorrent::download_priority_tag, void> > already registered; second conversion method ignored.
<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for boost::asio::ip::basic_endpoint<boost::asio::ip::tcp> already registered; second conversion method ignored.
<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for boost::asio::ip::basic_endpoint<boost::asio::ip::udp> already registered; second conversion method ignored.
<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for boost::asio::ip::address already registered; second conversion method ignored.
<frozen importlib._boots

### Mount Google Drive
To stream files we need to mount Google Drive.

In [3]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


### Add From Torrent File
You can run this cell to add more files as many times as you want

In [ ]:
from google.colab import files

source = files.upload()
params = {
    "save_path": "/content/drive/My Drive/Torrent",
    "ti": lt.torrent_info(list(source.keys())[0]),
}
downloads.append(ses.add_torrent(params))

### Add From Magnet Link
You can run this cell to add more files as many times as you want

In [4]:
params = {"save_path": "/content/drive/My Drive/Torrent"}

while True:
    magnet_link = input("Enter Magnet Link Or Type Exit: ")
    if magnet_link.lower() == "exit":
        break
    downloads.append(
        lt.add_magnet_uri(ses, magnet_link, params)
    )


Enter Magnet Link Or Type Exit: magnet:?xt=urn:btih:de4c01d0e3145b58859ab768bbec9f8438420419&dn=rutor.info_%D0%90%D1%80%D0%BA%D0%B5%D0%B9%D0%BD+%2F+Arcane%3A+League+of+Legends+%5BS01-02%5D+%282021-2024%29+WEB-DLRip-AVC+%D0%BE%D1%82+DoMiNo+%26+%D1%81%D0%B5%D0%BB%D0%B5%D0%B7%D0%B5%D0%BD%D1%8C+%7C+D+%7C+Iyuno-SDI+Group&tr=udp://opentor.net:6969&tr=http://retracker.local/announce
Enter Magnet Link Or Type Exit: magnet:?xt=urn:btih:d04a6d2dd32a07c6329b0bfb66fedcac16f79d1e&dn=rutor.info_%D0%93%D0%B0%D0%B4%D0%BA%D0%B8%D0%B9+%D1%8F+4+%2F+Despicable+Me+4+%282024%29+BDRip+%D0%BE%D1%82+MegaPeer+%7C+D&tr=udp://opentor.net:6969&tr=http://retracker.local/announce
Enter Magnet Link Or Type Exit: magnet:?xt=urn:btih:d04a6d2dd32a07c6329b0bfb66fedcac16f79d1e&dn=rutor.info_%D0%93%D0%B0%D0%B4%D0%BA%D0%B8%D0%B9+%D1%8F+4+%2F+Despicable+Me+4+%282024%29+BDRip+%D0%BE%D1%82+MegaPeer+%7C+D&tr=udp://opentor.net:6969&tr=http://retracker.local/announce
Enter Magnet Link Or Type Exit: magnet:?xt=urn:btih:900e97ec517

RuntimeError: missing info-hash from URI

### Start Download
Source: https://stackoverflow.com/a/5494823/7957705 and [#3 issue](https://github.com/FKLC/Torrent-To-Google-Drive-Downloader/issues/3) which refers to this [stackoverflow question](https://stackoverflow.com/a/6053350/7957705)

In [ ]:
import time
from IPython.display import display
import ipywidgets as widgets

state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]

layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()

            bar.description = " ".join(
                [
                    download.name(),
                    str(s.download_rate / 1000),
                    "kB/s",
                    state_str[s.state],
                ]
            )
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
            download_bars.remove(bar)
            print(download.name(), "complete")
    time.sleep(1)


FloatSlider(value=0.0, disabled=True, layout=Layout(width='auto'), step=0.01, style=SliderStyle(description_wi…

FloatSlider(value=0.0, disabled=True, layout=Layout(width='auto'), step=0.01, style=SliderStyle(description_wi…

FloatSlider(value=0.0, disabled=True, layout=Layout(width='auto'), step=0.01, style=SliderStyle(description_wi…

FloatSlider(value=0.0, disabled=True, layout=Layout(width='auto'), step=0.01, style=SliderStyle(description_wi…

Moana.2.2024.WEB-DLRip.ELEKTRI4KA.avi complete
Despicable.Me.4.2024.D.BDRip.1.46Gb.MegaPeer complete
Despicable.Me.4.2024.D.BDRip.1.46Gb.MegaPeer complete
